In [21]:
#|default_exp Modern_LSTM

In [22]:
#|export

import torch
from torch import nn

In [23]:
#|export

if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"

### Always check device

In [24]:
device

'cpu'

# 2 ways to do this:
### a. make a linear layer which will itself initialise weights and multiply them with the input tensor
### b. make parameters and manually write the code for matrix multiplication
 
## Although 1st one is more efficient and error free, going on with the 2nd one for learning purposes


## *Considering input (i.e. x) of batch size = 1 for simplicity

# READ ATTENTIVELY FOR THIS ONE

In [25]:
#When you create an nn.Module on the CPU, the .parameters() function points to the params on the CPU ONLY.
#So, if you create the params and move it to the GPU, the self.parameters() function will return an empty sequence

#1.Its better to create the nn.Module entirely (including the params) on the cpu and moving its instance to the GPU
#ORRRRR
#1.Move the params to the GPU during the forward pass of the cell.

#We'll go with the 1st to complicate things less and learn more, however in time I will update the forward pass to
#be carried on the GPU only.


In [26]:
#|export

#LSTM Cell
class LSTM_Cell(nn.Module):
    
    def __init__(self,nin,hidden):
        super(LSTM_Cell,self).__init__()
        self.input_size=nin
        self.hidden_size=hidden
        self.tanh=nn.Tanh()
         
        #input gate weight
        self.in_xt=nn.Parameter(torch.Tensor(self.hidden_size,self.input_size))
        self.in_xb=nn.Parameter(torch.Tensor(self.hidden_size))
        self.in_ht=nn.Parameter(torch.Tensor(self.hidden_size,self.hidden_size))
        self.in_Sigmoid=nn.Sigmoid()
        
        #cell memory gate weight
        self.cm_xt=nn.Parameter(torch.Tensor(self.hidden_size,self.input_size))
        self.cm_xb=nn.Parameter(torch.Tensor(self.hidden_size))
        self.cm_ht=nn.Parameter(torch.Tensor(self.hidden_size,self.hidden_size))
        self.cm_Tanh=nn.Tanh()
        
        #forget gate weights
        self.f_xt=nn.Parameter(torch.Tensor(self.hidden_size,self.input_size))
        self.f_xb=nn.Parameter(torch.Tensor(self.hidden_size))
        self.f_ht=nn.Parameter(torch.Tensor(self.hidden_size,self.hidden_size))
        self.f_Sigmoid=nn.Sigmoid()
        
        #output gate weights
        self.op_xt=nn.Parameter(torch.Tensor(self.hidden_size,self.input_size))
        self.op_xb=nn.Parameter(torch.Tensor(self.hidden_size))
        self.op_ht=nn.Parameter(torch.Tensor(self.hidden_size,self.hidden_size))
        self.op_Sigmoid=nn.Sigmoid()
        
        self.init_params()
        
        
    def init_params(self):
        for each_p in self.parameters():
            nn.init.uniform_(each_p, -1,1)
    
    def input_gate(self,x,h):
        inp=x @ self.in_xt.T + self.in_xb
        hidden=h @ self.in_ht.T
        return self.in_Sigmoid(inp+hidden)
    
    def cell_memory_gate(self,x,h):
        inp=x @ self.cm_xt.T + self.cm_xb
        hidden=h @ self.cm_ht.T
        return self.cm_Tanh(inp+hidden)
    
    def forget_gate(self,x,h):
        inp=x @ self.f_xt.T + self.f_xb
        hidden=h @ self.f_ht.T
        return self.f_Sigmoid(inp+hidden)
    
    def output_gate(self,x,h):
        inp=x @ self.op_xt.T + self.op_xb
        hidden=h @ self.op_ht.T
        return self.op_Sigmoid(inp+hidden)
    
    def forward(self,x,tuple_of_c_h):
        h_state_prev, c_state_prev = tuple_of_c_h
        
        #working through input gate
        inp=self.input_gate(x,h_state_prev)
        
        #working through the forget gate
        forget=self.forget_gate(x,h_state_prev)
        
        #working through the cell memory gate
        c_m=self.cell_memory_gate(x,h_state_prev)    
        
        #working through the ouput gate
        initial_op=self.output_gate(x,h)
        
        c_state_curr=forget*c_state_prev + inp*c_m
        
        h_state_curr=self.tanh(c_state_curr)*initial_op
        
        return (h_state_curr,c_state_curr)
     

In [27]:
#|export

#LSTM Layer
class LSTM_Layer(nn.Module):
    
    def __init__(self,nin,hidden):
        super(LSTM_Layer,self).__init__()
        self.input_size=nin
        self.hidden_size=hidden
        
        self.LSTM_cells=[LSTM_Cell(self.input_size,self.hidden_size) for _ in range(self.hidden_size)]
    
    def forward(self,x,tuple_of_c_h):
        
        h_state, c_state=tuple_of_c_h
        new_h_state=[]
        new_c_state=[]
        
        for i in range(self.hidden_size):
            
            
            cell=self.LSTM_cells[i]
            a,b=cell(x,(h_state[i],c_state[i]))
            new_h_state.append(a)
            new_c_state.append(b)
            
            return (new_h_state,new_c_state)
            
        

In [28]:
#|export

#LSTM Net
class LSTM_Net(nn.Module):
    
    def __init__(self,nin,hidden,layers):
        super(LSTM_Net,self).__init__()
        self.input_size=nin
        self.hidden_size=hidden
        self.layers=layers
        
        self.LSTM_Layers=[LSTM_Layer(self.input_size,self.hidden_size) if each==0 else LSTM_Layer(self.hidden_size,self.hidden_size) for each in range(self.layers)]
        
        #pre_final layer
        self.pre_final=nn.Linear(self.hidden_size,1)
        #final_layer
        self.final=nn.Linear(self.hidden_size,1,bias=False)
        
    def forward(x):
        #batch size of x is 1
        len_of_sentence,_=x.size()
        
        #initial hidden and cell states which will further get updated
        h_states=[torch.rand(self.hidden_size,self.hidden_size).to(device) for _ in range(self.layers)]
        c_states=[torch.rand(self.hidden_size,self.hidden_size).to(device) for _ in range(self.layers)]
        
        
        #working through time steps
        #time steps are same as the length of sentence, as you need to remember the entire sentence
        for time_step in range(len_of_sentence):
            #x at that time step
            x_t=x[time_step]
            
            for i in range(self.layers):
                
                layer=self.LSTM_Layers[i]
                
                h_states[i],c_states[i]=layer(x_t, (h_states[i],c_states[i]))
                x_t=h_states[i]
                                              
            #passing final hidden states through a linear layers
            out=[]
            for i in range(len(h_states[-1])):
                temp_var=self.pre_final(h_states[-1][i])
                out.append(temp_var)
            
            out=torch.Tensor(out)
            out=self.final(out)
            return out                   
        
        

In [29]:
#exporting notebook to vscode

from nbdev.export import nb_export
nb_export('developing_Modern_LSTM.ipynb','.')